In [1]:
import pandas as pd

dtype_map = {
    "customer_id": "int64",
    "email": "string",
    "first_name": "string",
    "last_name": "string",
    "phone": "string",
    "date_of_birth": "string",
    "gender": "string",
    "country": "string",
    "city": "string",
    "postal_code": "string",
    "address": "string",
    "registration_date": "string",
    "last_login": "string",
    "is_active": "string",
    "customer_segment": "string",
    "marketing_consent": "string",
}

df_customers = pd.read_csv("../data/raw/customers.csv", dtype=dtype_map)
df_customers.head(10)

,Unnamed: 0,customer_id,email,first_name,last_name,phone,date_of_birth,gender,country,city,postal_code,address,registration_date,last_login,is_active,customer_segment,marketing_consent
0,0,1,uthomas@example.net,Kayla,Smith,329-672-0449x89707,1948-05-20,F,Brazil,south michelle,70351,<NA>,2022-09-22T09:40:47.913063,2025-01-07T12:09:24.323425,True,Regular,True
1,1,2,caldwellandrew@example.com,Carolyn,Gray,(710)769-1350x042,1983-03-19,F,France,Lake Shaneville,65372,"247 Allison Overpass Suite 960 North Dillon, A...",2022-11-14T23:20:08.324356,2024-08-25T17:36:03.417619,True,Budget,False
2,2,3,jon32@example.org,Deborah,Martin,+1-694-797-6792,1940-05-23,M,USA,Lisaborough,32855,"61358 Mills Spur Lake Tiffany, MS 52073",2022-10-14T15:49:17.728606,2023-04-10T06:15:20.412198,True,budget,True
3,3,4,ksmith@example.org,Rachel,Ellison,<NA>,1969-12-21,M,Australia,south michaelborough,11530,undefined,2024-03-16T09:18:53.251566,2025-03-20T12:06:38.326651,True,undefined,False
4,4,5,andrewsjessica@example.net,Lisa,Phelps,001-800-821-0991,1940-05-18,M,Japan,Herrerabury,61228,"6184 King Trail Lake Trevor, IA 55611",2023-07-21T10:07:22.968497,2025-04-19T11:40:01.441135,True,Regular,True
5,5,6,dclements@example.org,Victoria,Eaton,576-743-1228x26213,<NA>,F,Australia,Lake Sandraville,49605,USCGC HOFFMAN FPO AP 15900,2023-08-17T17:48:25.714538,2024-10-13T12:03:14.459078,True,REGULAR,True
6,6,7,tylerdavis@example.net,Caitlin,Jordan,(535)967-4666x723,1983-05-17,M,France,Courtneymouth,<NA>,"1006 sergio greens port nathantown, dc 01974",2025-04-01T05:12:04.559047,2025-05-17T22:35:34.220988,True,Premium,True
7,7,8,<NA>,Elizabeth,Castro,739.558.8931x11106,1979-12-19,M,Germany,South Jennifer,94992,"01375 Barnes Junction Port John, FL 90700",2023-05-05T12:59:19.139965,undefined,True,Regular,False
8,8,9,lopezmelanie@example.org,Jane,Morris,886.375.3429x695,1957-10-10,F,Australia,Faithtown,18974,"9954 HARDIN THROUGHWAY CASTROBOROUGH, WY 66020",2024-04-01T04:43:01.999015,2025-06-13T08:27:38.284600,True,Regular,True
9,9,10,horneleslie@example.org,John,Williamson,(777)791-4118,1957-06-06,F,USA,Jonestown,28941,undefined,2022-08-17T14:47:16.736755,2024-11-16T10:14:44.823360,False,Regular,False


Vamos a descartar las columnas que son innecesarias para nuestro análisis.

In [2]:
df_customers.drop(
    columns=[
        "Unnamed: 0",
        # "first_name", Keep it for the second query of the proyect
        "email",
        "last_name",
        "phone",
        # "address", Keep it for the second query of the proyect
    ],
    inplace=True,
)

In [3]:
df_customers_clean = df_customers.set_index("customer_id")

# DATES

for col in ["date_of_birth", "registration_date", "last_login"]:
    df_customers_clean[col] = df_customers_clean[col].str.strip()
    df_customers_clean[col] = pd.to_datetime(df_customers_clean[col], errors="coerce")

# BOOLEANS

for col in ["is_active", "marketing_consent"]:
    df_customers_clean[col] = df_customers_clean[col].str.strip().str.lower()
    df_customers_clean[col] = (
        df_customers_clean[col]
        .map({"true": True, "false": False, "1": True, "0": False})
        .astype("boolean")
    )

# NARROWED CATEGORIES

valid_segments = ["Regular", "Premium", "Budget"]
df_customers_clean["customer_segment"] = (
    df_customers_clean["customer_segment"].str.strip().str.title()
)
df_customers_clean["customer_segment"] = df_customers_clean["customer_segment"].where(
    df_customers_clean["customer_segment"].isin(valid_segments)
)
#

valid_genders = ["M", "F"]
df_customers_clean["gender"] = df_customers_clean["gender"].str.strip().str.upper()
df_customers_clean["gender"] = df_customers_clean["gender"].where(
    df_customers_clean["gender"].isin(valid_genders)
)
df_customers_clean["gender"] = df_customers_clean["gender"].astype("category")

# CATEGORICAL

df_customers_clean["country"] = (
    df_customers_clean["country"].str.strip().str.title().astype("category")
)
df_customers_clean["city"] = (
    df_customers_clean["city"].str.strip().str.title().astype("category")
)

# STRINGS

df_customers_clean["postal_code"] = df_customers_clean["postal_code"].str.strip()
df_customers_clean["postal_code"] = df_customers_clean["postal_code"].where(
    df_customers_clean["postal_code"].str.match(r"^\d+$"), pd.NA
)
df_customers_clean["postal_code"] = df_customers_clean["postal_code"].astype("string")

df_customers_clean["first_name"] = df_customers_clean["first_name"].str.strip()

df_customers_clean.head()

,first_name,date_of_birth,gender,country,city,postal_code,address,registration_date,last_login,is_active,customer_segment,marketing_consent
customer_id,,,,,,,,,,,,
1,Kayla,1948-05-20,F,Brazil,South Michelle,70351,<NA>,2022-09-22 09:40:47.913063,2025-01-07 12:09:24.323425,True,Regular,True
2,Carolyn,1983-03-19,F,France,Lake Shaneville,65372,"247 Allison Overpass Suite 960 North Dillon, A...",2022-11-14 23:20:08.324356,2024-08-25 17:36:03.417619,True,Budget,False
3,Deborah,1940-05-23,M,Usa,Lisaborough,32855,"61358 Mills Spur Lake Tiffany, MS 52073",2022-10-14 15:49:17.728606,2023-04-10 06:15:20.412198,True,Budget,True
4,Rachel,1969-12-21,M,Australia,South Michaelborough,11530,undefined,2024-03-16 09:18:53.251566,2025-03-20 12:06:38.326651,True,<NA>,False
5,Lisa,1940-05-18,M,Japan,Herrerabury,61228,"6184 King Trail Lake Trevor, IA 55611",2023-07-21 10:07:22.968497,2025-04-19 11:40:01.441135,True,Regular,True


Exportamos el dataset limpio para reutilizarlo.

In [4]:
df_customers_clean.to_pickle("../data/clean/customers.pkl")